In [9]:
import numpy as np
import pandas as pd
import copy
import random
from util.util_load             import read_txt
from util.util_reschedule       import random_events, store_schedule, update_schedule, snapshot
from util.util_display          import plot, pretty_table
from env_action.metaheu         import GeneticAlgorithm
from env_action.action_space    import action_space

maxtime                = 3
maxtimeacceptnewjob    = 150
defectProb             = 0.03

action_name            = ["exact", "GA", "TS", "LFOH", "LAPH", "LAP_LFO", "CDR1", "CDR2", "CDR3", "CDR4", "CDR5", "CDR6"]

J, I, K, p_ijk, h_ijk,   \
d_j, n_j, MC_ji, n_MC_ji,\
OperationPool            = read_txt("DATA/jobs_small.txt")


variables = ['J', 'p_ijk', 'h_ijk', 'n_j', 'MC_ji', 'n_MC_ji']
for var in variables:
    globals()['org_' + var] = copy.deepcopy(globals()[var])

S_k                    = np.zeros((K))
S_j                    = np.zeros((J))
n_ops_left_j           = copy.deepcopy(n_j)
MB_info                = np.zeros((0))

t                      = 0
JSet                   = list(range(J))
OJSet                  = [[] for _ in range(J)]
for j in JSet:
    OJSet[j]           = [i for i in range(int(n_j[j]))]
Oij_on_machine         = [[] for _ in range(K)]

JA_event               = []
MB_event               = [[] for _ in range(K)]
events                 = {}
MB_record              = {}

# Preschedule
pre_GBest, X_ijk, S_ij, C_ij, C_j   = GeneticAlgorithm (S_k, S_j, JSet, OJSet, J, I, K, 
                                                        p_ijk, h_ijk, d_j, n_j, n_ops_left_j, 
                                                        MC_ji, n_MC_ji, OperationPool, maxtime= 1000)
print("preschedule")

fig1 = plot(J, K, n_j, X_ijk, S_ij, C_ij, MB_record, t)
display(fig1)

preschedule


In [10]:
print(S_ij[:, 10])
print(t)

[16. 18. 23.  0.  0.  0.  0.]
0


In [11]:
# Running schedule
completion = False
# Random events
JA_event, MB_event, events, t, trigger              = random_events(t, K, X_ijk, S_ij, C_ij, JA_event, MB_event, events,\
                                                                    maxtimeacceptnewjob, J, defectProb)
while completion == False and trigger is not None:
    if t < np.max(C_ij):
        # Snapshot
        S_k, S_j, J, I, JSet, OJSet, DSet, ODSet,    \
        OperationPool, n_ops_left_j, MC_ji, n_MC_ji, \
        d_j, n_j, p_ijk, h_ijk,                      \
        org_p_ijk, org_h_ijk, org_n_j,               \
        org_MC_ji, org_n_MC_ji,                      \
        Oij_on_machine, affected_Oij,                \
        MB_info, MB_record,  X_ijk, S_ij, C_ij,      \
        JA, percent_JA, MB, percent_MB, remaining_re,\
        CT_k, T_cur, Tard_job, C_j                   = snapshot(t, trigger, MC_ji, n_MC_ji,                         \
                                                                d_j, n_j, p_ijk, h_ijk, J, I, K, X_ijk, S_ij, C_ij, \
                                                                OperationPool, MB_info, MB_record, S_k,             \
                                                                org_J, org_p_ijk, org_h_ijk, org_n_j,               \
                                                                org_MC_ji, org_n_MC_ji, C_j                         )
        # Store previous schedule
        X_previous, S_previous, C_previous           = store_schedule(X_ijk, S_ij, C_ij)       
        action_method 						         = action_space(J, I, K, p_ijk, h_ijk, d_j, n_j, 
                                                                    MC_ji, n_MC_ji, n_ops_left_j, OperationPool, S_k, S_j, 
                                                                    JSet, OJSet, Oij_on_machine, affected_Oij, 
                                                                    t, X_ijk, S_ij, C_ij, C_j, CT_k, T_cur, Tard_job, maxtime)
        action_id                                    = 2
        print(action_name[action_id])

        reschedule							         = action_method[action_id]
        GBest, X_ijk, S_ij, C_ij, C_j                = reschedule()
        print(GBest)
        X_ijk, S_ij, C_ij                            = update_schedule(DSet, ODSet, X_ijk, S_ij, C_ij, X_previous, S_previous, C_previous)
        fig2 = plot(J, K, n_j, X_ijk, S_ij, C_ij, MB_record, t)
        display(fig2)
        

        JA_event, MB_event, events, t, trigger      = random_events(t, K, X_ijk, S_ij, C_ij, JA_event, MB_event, events,\
                                                                    maxtimeacceptnewjob, J, defectProb)
        completion = True
    # else: completion = True


TS
1 3 1
job 1 operation 2
check k 12
9 7 4
job 9 operation 3
check k 21
4 6 1
job 4 operation 5
check k 29
5 6 4
job 5 operation 2
check k 20
13 6 3
job 13 operation 3
check k 22
job 13 operation 4
check k 26
7 3 1
job 7 operation 2
check k 12
17 7 4
job 17 operation 3
check k 21
job 9 operation 4
check k 22
job 5 operation 3
check k 24
16 7 4
job 16 operation 3
check k 20
job 9 operation 5
check k 25
job 5 operation 4
check k 26
job 13 operation 5
check k 27
job 16 operation 4
check k 24
job 17 operation 4
check k 24
job 5 operation 5
check k 28
job 16 operation 5
check k 26
job 16 operation 6
check k 27
job 17 operation 5
check k 26
job 9 operation 6
check k 29
job 17 operation 6
check k 29
11.0


In [12]:
print(t)
print(S_ij[:, 9])
print(n_j[9], n_ops_left_j[9])

70
[  0.   5.  11.  70. 100. 116. 143.]
7 4
